In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn.neighbors import LocalOutlierFactor
from pandas_profiling import ProfileReport

In [2]:
data_path = '../data/'
output_path = '../out/'

In [3]:
data = pd.read_csv(data_path + 'bikes.csv')

/home/marek-szyd/Programs/anaconda3/envs/dv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data.drop(
    [
        'departure_latitude', 'departure_longitude', 'return_latitude', 'return_longitude',
        'departure_id', 'return_id'
    ],
    axis=1, inplace=True
) 

In [5]:
data.head()

,departure,return,departure_name,return_name,distance (m),duration (sec.),avg_speed (km/h),Air temperature (degC)
0,2020-03-23 06:09:44,2020-03-23 06:16:26,Kuusitie,Esterinportti,1747.0,401.0,0.261397,0.9
1,2020-03-23 06:11:58,2020-03-23 06:26:31,Kamppi (M),Kasarmitori,1447.0,869.0,0.099908,0.9
2,2020-03-23 06:16:29,2020-03-23 06:24:23,Porolahden koulu,Agnetankuja,1772.0,469.0,0.226695,0.9
3,2020-03-23 06:33:53,2020-03-23 07:14:03,Vallipolku,Korppaanmäentie,7456.0,2406.0,0.185935,0.9
4,2020-03-23 06:36:09,2020-03-23 07:04:10,Länsisatamankatu,Vilhonvuorenkatu,7120.0,1679.0,0.254437,0.9


In [6]:
data.describe()

,distance (m),duration (sec.),avg_speed (km/h),Air temperature (degC)
count,1.215746e+07,1.215746e+07,1.215391e+07,1.214156e+07
mean,2.295275e+03,9.597751e+02,3.355556e-01,1.565044e+01
std,2.452067e+04,7.346528e+03,3.428006e+01,5.497952e+00
min,-4.292467e+06,0.000000e+00,-4.689001e+02,-5.200000e+00
25%,1.000000e+03,3.440000e+02,1.467403e-01,1.230000e+01
50%,1.739000e+03,5.860000e+02,1.863679e-01,1.640000e+01
75%,2.869000e+03,9.710000e+02,2.204348e-01,1.930000e+01
max,3.681399e+06,5.401659e+06,1.699104e+04,3.290000e+01


In [7]:
data.columns

Index(['departure', 'return', 'departure_name', 'return_name', 'distance (m)',
       'duration (sec.)', 'avg_speed (km/h)', 'Air temperature (degC)'],
      dtype='object')

## Rename column names

In [5]:
data.rename(columns={
    'distance (m)' : 'distance',
    'duration (sec.)' : 'duration',
    'avg_speed (km/h)' : 'avg_speed',
    'Air temperature (degC)' : 'temperature'
}, inplace=True)

In [9]:
data.dtypes

departure          object
return             object
departure_name     object
return_name        object
distance          float64
duration          float64
avg_speed         float64
temperature       float64
dtype: object

## Change datatypes to dates

In [6]:
data['departure'] = pd.to_datetime(data['departure'])
data['return'] = pd.to_datetime(data['return'])

In [11]:
data.head(10)

,departure,return,departure_name,return_name,distance,duration,avg_speed,temperature
0,2020-03-23 06:09:44,2020-03-23 06:16:26,Kuusitie,Esterinportti,1747.0,401.0,0.261397,0.9
1,2020-03-23 06:11:58,2020-03-23 06:26:31,Kamppi (M),Kasarmitori,1447.0,869.0,0.099908,0.9
2,2020-03-23 06:16:29,2020-03-23 06:24:23,Porolahden koulu,Agnetankuja,1772.0,469.0,0.226695,0.9
3,2020-03-23 06:33:53,2020-03-23 07:14:03,Vallipolku,Korppaanmäentie,7456.0,2406.0,0.185935,0.9
4,2020-03-23 06:36:09,2020-03-23 07:04:10,Länsisatamankatu,Vilhonvuorenkatu,7120.0,1679.0,0.254437,0.9
5,2020-03-23 06:37:52,2020-03-23 06:58:56,Radiokatu,Porthania,5169.0,1262.0,0.245753,0.9
6,2020-03-23 06:39:51,2020-03-23 06:45:30,Tyynenmerenkatu,Hietalahdentori,1194.0,335.0,0.213851,0.9
7,2020-03-23 06:44:37,2020-03-24 12:02:19,Itämerentori,Meilahden sairaala,3651.0,1195.0,0.183314,0.9
8,2020-03-23 06:47:18,2020-03-23 07:01:09,Koskelantie,Kalasatama (M),3120.0,827.0,0.226360,0.9
9,2020-03-23 06:47:22,2020-03-23 06:56:22,Rautatieläisenkatu,A.I. Virtasen aukio,2070.0,535.0,0.232150,0.9


## Multiply avg_speed column by 100 to get meaningful values

In [7]:
data['avg_speed'] *= 100

In [14]:
data_len = len(data.index)
print(f"Length of the dataset before removal of usunal data points: {data_len}")

data.drop(data[((data.distance <= 0) | (data.avg_speed <= 0))].index, inplace=True)

Length of the dataset before removal of usunal data points: 12157458


In [15]:
quantiles = data[['distance', 'duration', 'avg_speed']].quantile([0.05, 0.25, 0.5, 0.75, 0.95])
quantiles

,distance,duration,avg_speed
0.05,453.0,164.0,5.671378
0.25,1057.0,360.0,15.166987
0.50,1794.0,600.0,18.844737
0.75,2924.0,983.0,22.166667
0.95,5298.0,1680.0,26.868750


In [16]:
for col in ['distance', 'duration', 'avg_speed']:
    data.drop(data[
        (data[col] < quantiles.loc[0.05][col]) | (data[col] > quantiles.loc[0.95][col])
    ].index, inplace=True)

data.reset_index(drop=True, inplace=True)
print(f"Length of the dataset after removal of usunal data points: {len(data.index)}")
print(f"Removed {data_len - len(data.index)} entries.")

Length of the dataset after removal of usunal data points: 9604122
Removed 2553336 entries.


## Detect outliers from the dataset and remove them

In [20]:
# Map -1 to True and 1 to False for outliers
converter = lambda val: True if val == -1 else False

In [21]:
# !!! The outlier detection takes a lot of RAM and time and depends on your hardware.

clf = LocalOutlierFactor(n_neighbors=30, n_jobs=-1)
outlier_array = clf.fit_predict(data[['distance', 'duration', 'avg_speed']])

outliers = np.array([converter(val) for val in outlier_array])

In [22]:
print(f"Detected {len(data[outliers])} ({round(len(data[outliers]) / len(data) * 100, 2)}%) outliers in the dataset!")
data[outliers].head(20)

Detected 285739 (2.98%) outliers in the dataset!


,departure,return,departure_name,return_name,distance,duration,avg_speed,temperature
8,2020-03-23 06:53:10,2020-03-23 07:00:38,Töölönkatu,Rautatientori / itä,1418.0,447.0,0.190336,0.9
80,2020-03-23 08:19:21,2020-03-23 08:23:02,Leppävaaran urheilupuisto,Muurarinkuja,646.0,220.0,0.176182,1.5
111,2020-03-23 08:46:52,2020-03-23 08:52:05,Herttoniemi (M),Laivalahden puistotie,808.0,309.0,0.156893,1.5
158,2020-03-23 09:32:37,2020-03-23 09:39:10,Intiankatu,Paavalinpuisto,1190.0,388.0,0.184021,1.9
322,2020-03-23 12:00:41,2020-03-23 12:07:44,Kamppi (M),Perämiehenkatu,1176.0,418.0,0.168804,3.3
353,2020-03-23 12:23:57,2020-03-23 12:29:16,Laivalahden puistotie,Herttoniemi (M),810.0,316.0,0.153797,3.3
370,2020-03-23 12:36:34,2020-03-23 12:42:25,Töölönkatu,Mannerheimintie,1160.0,349.0,0.199427,3.3
371,2020-03-23 12:36:50,2020-03-23 12:42:32,Töölönkatu,Mannerheimintie,1152.0,341.0,0.202698,3.3
373,2020-03-23 12:37:29,2020-03-23 12:44:15,Sammonpuistikko,Lastenlehto,1092.0,403.0,0.162581,3.3
412,2020-03-23 13:03:40,2020-03-23 13:07:04,Kauppakorkeakoulu,Kansallismuseo,615.0,202.0,0.182673,4.0


In [23]:
print(f"Data before removal of outliers has {len(data.index)} rows.")
data.drop(data[outliers].index, inplace=True)
data.reset_index(drop=True, inplace=True)
print(f"Data after removal of outliers has {len(data.index)} rows.")

Data before removal of outliers has 9604122 rows.
Data after removal of outliers has 9318383 rows.


## Create HTML report of the dataset in out/ directory

In [17]:
# !!! Full profiling takes a lot of time and big chunk of RAM; Uncomment the next line to run full profiler that includes correlations between attributes

# profile = ProfileReport(data, title='Helsinki City Bikes').to_file(output_path + 'full_bikes.html')
ProfileReport(data, title='Helsinki City Bikes', minimal=True).to_file(output_path + 'partial_bikes.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 932.69it/s]


In [18]:
data.groupby(by='departure_name').size().sort_values(ascending=False)

departure_name
Itämerentori              269862
Töölönlahdenkatu          202555
Kamppi (M)                164966
Rautatientori / länsi     143564
Ympyrätalo                140941
                           ...  
Kutsuntatie                 1734
Haakoninlahdenkatu          1718
Itäkeskus Metrovarikko      1083
Outotec                      814
Derby Business Park          243
Length: 347, dtype: int64

In [8]:
def replace_comma(val: str):
    return ' / '.join(val.split(', '))

replaced = data[data['departure_name'].str.contains(',')]['departure_name'].map(replace_comma)
data.loc[replaced.index, 'departure_name'] = replaced

## Export unique station names to file

In [19]:
pd.Series(data['departure_name'].unique()).to_csv(data_path + 'stations.csv', header=['name'])

## Export cleaned dataset to file

In [22]:
data.to_csv(path_or_buf=data_path + 'reduced.csv', index=False)